In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from patsy import dmatrices
import statsmodels.api as sm;
from statsmodels.stats.outliers_influence import variance_inflation_factor
%matplotlib inline
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [14]:
import time
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [4]:
info_df_raw = pd.read_csv("/content/drive/My Drive/데이터분석/samsung/info1.csv")
feat_df_raw = pd.read_csv("/content/drive/My Drive/데이터분석/samsung/samp_cst_feat.csv")
train_df_raw = pd.read_csv("/content/drive/My Drive/데이터분석/samsung/samp_train.csv")
dtype_df_raw = pd.read_excel("/content/drive/My Drive/데이터분석/samsung/variable_dtype.xlsx")

In [5]:
info_df = info_df_raw.copy()
feat_df = feat_df_raw.copy()
train_df = train_df_raw.copy()

In [6]:
dtype_dict = dtype_df_raw.set_index('Variable_Name')['dType'].to_dict()

In [7]:
#카테고리컬 변수타입 변환
for w in feat_df_raw.columns:
  if w in dtype_dict.keys():
    if dtype_dict[w] == "categorical":
      feat_df[w] = feat_df_raw[w].astype(str)
feat_df.dtypes.head(10)

cst_id_di      int64
VAR002       float64
VAR003       float64
VAR004       float64
VAR005       float64
VAR006       float64
VAR007        object
VAR008       float64
VAR009       float64
VAR010       float64
dtype: object

In [8]:
#카테고리컬 변수들 행 이름
obj_col = []
numerical_col = [] 
for w in dtype_dict:
  if dtype_dict[w] == "categorical":
    obj_col.append(w)
  else:
    numerical_col.append(w)

In [9]:
##categorical은 다 dummy variables
for i in obj_col:
  print(feat_df[i].value_counts())
  print()

0    8633
1    1491
Name: VAR007, dtype: int64

0    5430
1    4694
Name: VAR015, dtype: int64

0    8009
1    2115
Name: VAR018, dtype: int64

0    9926
1     198
Name: VAR026, dtype: int64

0    7606
1    2518
Name: VAR059, dtype: int64

0    7324
1    2800
Name: VAR066, dtype: int64

1    8723
0    1401
Name: VAR067, dtype: int64

0    8166
1    1958
Name: VAR070, dtype: int64

0    9314
1     810
Name: VAR071, dtype: int64

0    8769
1    1355
Name: VAR077, dtype: int64

0    9738
1     386
Name: VAR078, dtype: int64

0    9515
1     609
Name: VAR094, dtype: int64

0    9158
1     966
Name: VAR096, dtype: int64

0    8956
1    1168
Name: VAR097, dtype: int64

0    9306
1     818
Name: VAR098, dtype: int64

0    8434
1    1690
Name: VAR107, dtype: int64

0    7949
1    2175
Name: VAR111, dtype: int64

0    7452
1    2672
Name: VAR124, dtype: int64

0    7249
1    2875
Name: VAR127, dtype: int64

0    9952
1     172
Name: VAR143, dtype: int64

0    9485
1     639
Name: VAR144, dtype:

In [56]:
total_df = feat_df_raw.copy()
total_df['label'] = train_df_raw['MRC_ID_DI']

In [57]:
df = total_df.copy()

In [58]:
X = df.drop(['label'], axis = 1)
Y = df['label']
X = StandardScaler().fit_transform(X)

In [59]:
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
X_pca = pca.fit_transform(X)
PCA_df = pd.DataFrame(data = X_pca, columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
PCA_df = pd.concat([PCA_df, df['label']], axis = 1)
PCA_df.head()

,PC1,PC2,PC3,PC4,PC5,label
0,7.279507,-3.813251,5.084972,-5.013252,-0.228353,0
1,14.681195,4.586905,-2.685685,2.918950,-3.614974,8
2,-14.916882,-1.823236,0.040112,-1.621080,-5.169928,0
3,-3.165472,4.684853,-3.915588,-4.432158,1.878737,5
4,19.599368,7.075075,4.594998,2.728054,-0.535296,6


In [60]:
print(pca.explained_variance_)

[85.45969829 23.29474153 20.62239031 12.33915741  7.18065995]


In [61]:
Y.values

array([0, 8, 0, ..., 0, 0, 0])

In [62]:
import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [92]:
x_train, x_test, y_train, y_test = train_test_split(total_df.drop("label",axis=1), total_df["label"], test_size = 0.30, random_state = 2020)

In [79]:
batch = 15
learning_rate = 0.0001
model = Sequential()
model.add(Dense(64, input_dim=5, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(11, activation='softmax'))

In [80]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [81]:
hist = model.fit(x_train, y_train, epochs=1000, batch_size=batch)

Epoch 1/1000


ValueError: ignored

In [82]:
!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc1

  Cloning https://github.com/keras-team/keras-tuner.git (to revision 1.0.2rc1) to /tmp/pip-req-build-0rtd2mou
  Running command git clone -q https://github.com/keras-team/keras-tuner.git /tmp/pip-req-build-0rtd2mou
  Running command git checkout -q 0fb69434a132093518e0e53d40020145ae192629
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2rc1-cp36-none-any.whl size=85424 sha256=76591834eeedfe7a79679b094f139038937ac42aa8ed374e2acd82dd10ba42cc
  Stored in directory: /tmp/pip-ephem-wheel-cache-7nq68zt3/wheels/af/c9/7c/6ea01f9753a5dd1484136b4cb7b33a0a7fba253e5c74ade5af
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=833595f689a3975009cba005b73bc86d70d490f025ad32fa70f364c4e509c41c
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [83]:
import tensorflow as tf
import autokeras as ak

In [94]:
clf = ak.StructuredDataClassifier(
    overwrite=True,
    max_trials=3)
# Feed the structured data classifier with training data.
clf.fit(x_train, y_train, epochs=150)
# Predict with the best model.
predicted_y = clf.predict(x_test)
# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

Trial 3 Complete [00h 02m 09s]
val_accuracy: 0.8021582961082458

Best val_accuracy So Far: 0.8021582961082458
Total elapsed time: 00h 06m 09s
INFO:tensorflow:Oracle triggered exit
Epoch 1/150
222/222 [==============================] - 1s 5ms/step - loss: 209833488.0000 - accuracy: 0.6689
Epoch 2/150
222/222 [==============================] - 1s 6ms/step - loss: 70796544.0000 - accuracy: 0.6741
Epoch 3/150
222/222 [==============================] - 1s 6ms/step - loss: 68285184.0000 - accuracy: 0.6706
Epoch 4/150
222/222 [==============================] - 1s 6ms/step - loss: 65767884.0000 - accuracy: 0.6715
Epoch 5/150
222/222 [==============================] - 1s 6ms/step - loss: 73603680.0000 - accuracy: 0.6689
Epoch 6/150
222/222 [==============================] - 1s 6ms/step - loss: 61936220.0000 - accuracy: 0.6693
Epoch 7/150
222/222 [==============================] - 1s 6ms/step - loss: 67066796.0000 - accuracy: 0.6751
Epoch 8/150
222/222 [==============================] - 1s 6ms/s